In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import os

import numpy as np
import rasterio
from tqdm import tqdm
from rasterio.errors import RasterioIOError

import sys
sys.path.append('../src')
from classes.data.satellite_image import SatelliteImage
from classes.labelers.labeler import Labeler
from utils.filter import (
    has_cloud, is_too_black2, mask_full_cloud, patch_nocloud
)


In [5]:
from train_pipeline_utils.download_data import load_satellite_data
from classes.data.satellite_image import SatelliteImage

In [30]:
load_satellite_data(2018, "974", "PLEIADES")

Entre dans la fonction load_satellite_data
download PLEIADES 974 2018 in /home/onyxia/work/detection-habitat-spontane/notebooks/../data/PLEIADES/2018/REUNION


'/home/onyxia/work/detection-habitat-spontane/notebooks/../data/PLEIADES/2018/REUNION'

In [62]:
def create_doss_cloud(year, dep):
    file_path = '../data/PLEIADES/' + year + "/" + dep
    output_masks_path = '../nuagespleiades/PLEIADES/' + year + "/" + dep

    if os.path.exists(output_masks_path):
        print("fichiers déjà écrits")
    
    if not os.path.exists(output_masks_path):
        os.makedirs(output_masks_path)
    
    
    list_name = os.listdir(file_path)
    list_path = [file_path + "/" + name for name in list_name]
    
    
    for path, file_name in tqdm(zip(list_path, list_name), total=len(list_path), desc='Processing'):
        try:
            big_satellite_image = SatelliteImage.from_raster(
                file_path=path, dep=None, date=None, n_bands=3
            )
            
        except RasterioIOError:
            continue
    
        else:
            boolean = has_cloud(big_satellite_image)
    
            if boolean:
                mask_full = mask_full_cloud(big_satellite_image)
                file_name = file_name.split(".")[0]
                np.save(output_masks_path + "/" + file_name + ".npy", mask_full)

    return(output_masks_path)

In [63]:
output_directory_name = create_doss_cloud("2022", "GUYANE")

Processing: 100%|██████████| 240/240 [04:10<00:00,  1.04s/it]


In [64]:
list_nuages = os.listdir(output_directory_name)
len(list_nuages)

90

In [73]:
from train_pipeline_utils.download_data import *

In [74]:
load_donnees_test(type="segmentation")

'/home/onyxia/work/detection-habitat-spontane/notebooks/../donnees-test/segmentation'

export MC_HOST_s3=https://8RXDFZKNQUVILN4E7WQY:AvnwPASHzev2yebxkVDOmZBwsZAiNL26zB9WPyxH:eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJhY2Nlc3NLZXkiOiI4UlhERlpLTlFVVklMTjRFN1dRWSIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sImF1ZCI6WyJtaW5pby1kYXRhbm9kZSIsIm9ueXhpYSIsImFjY291bnQiXSwiYXV0aF90aW1lIjoxNjg2NTUzOTY5LCJhenAiOiJvbnl4aWEiLCJlbWFpbCI6InJheWEuYmVyb3ZhQGVsZXZlLmVuc2FpLmZyIiwiZW1haWxfdmVyaWZpZWQiOnRydWUsImV4cCI6MTY4NjY2MTM4NiwiZmFtaWx5X25hbWUiOiJCRVJPVkEiLCJnaXZlbl9uYW1lIjoiUmF5YSIsImdyb3VwcyI6WyJmdW5hdGhvbiIsInNsdW1zLWRldGVjdGlvbiJdLCJpYXQiOjE2ODY1NTQxODEsImlzcyI6Imh0dHBzOi8vYXV0aC5sYWIuc3NwY2xvdWQuZnIvYXV0aC9yZWFsbXMvc3NwY2xvdWQiLCJqdGkiOiIzMzkyNmEwZC1mMjg2LTQ3MzMtODk2OC1hMzljOTFhY2Q1MzUiLCJsb2NhbGUiOiJlbiIsIm5hbWUiOiJSYXlhIEJFUk9WQSIsIm5vbmNlIjoiZDU1NjAxMzctNTVjYy00Y2I3LTk0MDAtOTNlMDFjNWJhNzY2IiwicG9saWN5Ijoic3Rzb25seSIsInByZWZlcnJlZF91c2VybmFtZSI6ImlkMTc1MyIsInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJvZmZsaW5lX2FjY2VzcyIsInVtYV9hdXRob3JpemF0aW9uIiwiZGVmYXVsdC1yb2xlcy1zc3BjbG91ZCJdfSwicmVzb3VyY2VfYWNjZXNzIjp7ImFjY291bnQiOnsicm9sZXMiOlsibWFuYWdlLWFjY291bnQiLCJtYW5hZ2UtYWNjb3VudC1saW5rcyIsInZpZXctcHJvZmlsZSJdfX0sInNjb3BlIjoib3BlbmlkIHByb2ZpbGUgZ3JvdXBzIGVtYWlsIiwic2Vzc2lvblBvbGljeSI6ImV5SldaWEp6YVc5dUlqb2lNakF4TWkweE1DMHhOeUlzSWxOMFlYUmxiV1Z1ZENJNlczc2lSV1ptWldOMElqb2lRV3hzYjNjaUxDSkJZM1JwYjI0aU9sc2ljek02S2lKZExDSlNaWE52ZFhKalpTSTZXeUpoY200NllYZHpPbk16T2pvNmNISnZhbVYwTFhOc2RXMXpMV1JsZEdWamRHbHZiaUlzSW1GeWJqcGhkM002Y3pNNk9qcHdjbTlxWlhRdGMyeDFiWE10WkdWMFpXTjBhVzl1THlvaVhYMHNleUpGWm1abFkzUWlPaUpCYkd4dmR5SXNJa0ZqZEdsdmJpSTZXeUp6TXpwTWFYTjBRblZqYTJWMElsMHNJbEpsYzI5MWNtTmxJanBiSW1GeWJqcGhkM002Y3pNNk9qb3FJbDBzSWtOdmJtUnBkR2x2YmlJNmV5SlRkSEpwYm1kTWFXdGxJanA3SW5Nek9uQnlaV1pwZUNJNkltUnBabVoxYzJsdmJpOHFJbjE5ZlN4N0lrVm1abVZqZENJNklrRnNiRzkzSWl3aVFXTjBhVzl1SWpwYkluTXpPa2RsZEU5aWFtVmpkQ0pkTENKU1pYTnZkWEpqWlNJNld5SmhjbTQ2WVhkek9uTXpPam82S2k5a2FXWm1kWE5wYjI0dktpSmRmVjE5Iiwic2Vzc2lvbl9zdGF0ZSI6ImRhOTVhMzdjLTljYTEtNGI4OS04ODdjLWI3YzYwNjhhNjE0YyIsInNpZCI6ImRhOTVhMzdjLTljYTEtNGI4OS04ODdjLWI3YzYwNjhhNjE0YyIsInN1YiI6ImRiNWM5MTg3LWUwNjItNDIyMy04N2UwLTk2YTc5ODYzMDZlOSIsInR5cCI6IkJlYXJlciJ9.HXiJ-VKBFzI5rH8-5bRbCqVgt4bbdiD6w7VbLxtUT4Iw4RB1QDQDSIUj5TqVTxy19bL3eF54sap9HSySVZFNLQ@minio.lab.sspcloud.fr

!mc cp --recursive nuagespleiades/PLEIADES/2022/GUYANE s3/projet-slums-detection/Donnees/NUAGESPLEIADES/2022/GUYANE/

In [114]:
import gc
import os
import sys
from datetime import datetime
from tqdm import tqdm
import mlflow
import numpy as np
import pytorch_lightning as pl
import torch
import yaml
from pytorch_lightning.callbacks import (
    EarlyStopping,
    LearningRateMonitor,
    ModelCheckpoint,
)
from torch.utils.data import DataLoader
from yaml.loader import SafeLoader

from classes.data.satellite_image import SatelliteImage
from classes.labelers.labeler import BDTOPOLabeler, RILLabeler
from classes.optim.losses import CrossEntropySelfmade
from torch.nn import CrossEntropyLoss
from classes.optim.optimizer import generate_optimization_elements
from data.components.dataset import PleiadeDataset
from models.components.segmentation_models import DeepLabv3Module
from models.segmentation_module import SegmentationModule
from train_pipeline_utils.download_data import load_satellite_data, load_donnees_test
from train_pipeline_utils.handle_dataset import (
    generate_transform,
    select_indices_to_split_dataset
)

from train_pipeline_utils.prepare_data import(
    filter_images,
    label_images,
    save_images_and_masks,
    check_labelled_images
)

from classes.data.satellite_image import SatelliteImage
from classes.data.labeled_satellite_image import SegmentationLabeledSatelliteImage
from utils.utils import update_storage_access, split_array, remove_dot_file
from rasterio.errors import RasterioIOError
from classes.optim.evaluation_model import evaluer_modele_sur_jeu_de_test_segmentation_pleiade
from run_training_pipeline import *

In [ ]:
# Open the file and load the file
with open("../config.yml") as f:
    config = yaml.load(f, Loader=SafeLoader)

list_data_dir, list_masks_cloud_dir, test_dir = download_data(config)

# list_data_dir = ["../data/PLEIADES/2022/MARTINIQUE"]
# list_masks_cloud_dir = ["../data/NUAGESPLEIADES/2022/MARTINIQUE"]

list_output_dir = prepare_train_data(config, list_data_dir, list_masks_cloud_dir)
prepare_test_data(config, test_dir)

model = instantiate_model(config)

train_dl, valid_dl, test_dl = instantiate_dataloader(
    config, list_output_dir
)

Entre dans la fonction download_data
Entre dans la fonction load_satellite_data
download NUAGESPLEIADES 972 2022 in /home/onyxia/work/detection-habitat-spontane/data/NUAGESPLEIADES/2022/MARTINIQUE
Entre dans la fonction load_satellite_data
download PLEIADES 972 2022 in /home/onyxia/work/detection-habitat-spontane/data/PLEIADES/2022/MARTINIQUE
chargement des données test
Entre dans la fonction prepare_data
Entre dans la fonction check_labelled_images
The directory doesn't exist and is going to be created.
Directory created


  3%|▎         | 37/1426 [01:29<1:05:02,  2.81s/it]